In [1]:
.L ../libAmBe.so

In [2]:
test

(const int) 11


In [3]:
char const* path {"../dataset/ambe2022.root"};
DataSetHolder dataset = ambeDataSet(path);

In [4]:
double All = dataset.df.Count().GetValue();

In [5]:
ROOT::RDF::RNode df = filterT1Events(dataset.df, 10e-6).Filter("!is_event", "not T1 Filter");

In [6]:
double NotT1 = df.Count().GetValue();
double T1 = All - NotT1;

In [29]:
All

(double) 196437.00


In [30]:
NotT1

(double) 77376.000


In [7]:
NotT1 / All * 100

(double) 39.389728


In [8]:
T1 / All * 100

(double) 60.610272


In [9]:
ROOT::RDF::RNode df2 = df.Define("volidNoise", "volidEpvtTot(volid, E_quenched)");
df2 = df2.Define("edep_pvtNoise", "std::get<1>(volidNoise)");
df2 = df2.Redefine("volidNoise", "std::get<0>(volidNoise)");

In [10]:
df2 = df2.Filter("ROOT::VecOps::Sum(edep_pvtNoise < 4.2 && edep_pvtNoise > 0.1) == 2", "only 2 cube");

In [11]:
double Elimits = df2.Count().GetValue();

In [12]:
Elimits / All * 100

(double) 8.9153265


In [13]:
Elimits / NotT1 * 100

(double) 22.633633


In [14]:
Elimits / T1 * 100

(double) 14.709267


In [15]:
ROOT::RDF::RNode df3 = df2.Redefine("volidNoise", "volidNoise[edep_pvtNoise < 4.2 && edep_pvtNoise > 0.1]");
df3 = df2.Redefine("edep_pvtNoise", "edep_pvtNoise[edep_pvtNoise < 4.2 && edep_pvtNoise > 0.1]");

In [16]:
df3 = df3.Define("dx", "getCubeX(volidNoise[0]) - getCubeX(volidNoise[1])");
df3 = df3.Define("dy", "getCubeY(volidNoise[0]) - getCubeY(volidNoise[1])");
df3 = df3.Define("dz", "getCubeZ(volidNoise[0]) - getCubeZ(volidNoise[1])");

In [17]:
df3 = df3.Define("distance", "std::sqrt(dx * dx + dy * dy + dz * dz)");

In [18]:
df3 = df3.Filter("distance >= 1.", "distance >= 1");

In [19]:
double NoZeroDist = df3.Count().GetValue();

In [20]:
NoZeroDist / All * 100

(double) 8.6068307


In [21]:
NoZeroDist / NotT1 * 100

(double) 21.850445


In [22]:
NoZeroDist / T1 * 100

(double) 14.200284


In [23]:
ROOT::RDF::RNode df4 = df3.Define("DeltaE", "std::abs(edep_pvtNoise[0] - edep_pvtNoise[1])");

In [24]:
df4 = df4.Filter("DeltaE > 3.7", "1 ~4.2 && 1 ~0.2");

In [25]:
double GoodDeltaE = df4.Count().GetValue();

In [31]:
GoodDeltaE

(double) 201.00000


In [26]:
GoodDeltaE / All * 100

(double) 0.10232288


In [27]:
GoodDeltaE / NotT1 * 100

(double) 0.25977047


In [28]:
GoodDeltaE / T1 * 100

(double) 0.16882102
